In [37]:
import json
import pandas as pd

with open('../panel_app/example_data/routing_tomtom_out.json') as f:
    response_json = json.load(f)
list_df_pts = []
leg_list= response_json['routes'][0]['legs']
for i in range(len(leg_list)):
    df = pd.DataFrame(leg_list[i]['points'])
    df['leg'] = i
    list_df_pts.append(df)
df_all_pts = pd.concat(list_df_pts)

DEFAULT_DEST = [
    "1122 3 St SE",
    "1706 11 ave sw",
    "445 Marsh Rd NE",
    "504 Berkley Cres NW",
    "880 Center Ave NE",
    "111 25 Ave SW",
    "7016 34 Ave NW",
    "2313 15A st SE",
]

DEFAULT_LATLONG = [(51.04191, -114.05572),
 (51.04357, -114.0996),
 (51.05276, -114.05288),
 (51.12546, -114.08387),
 (51.05257, -114.04246),
 (51.03027, -114.06479),
 (51.08401, -114.18789),
 (51.03297, -114.02386)]



In [38]:
df_label = pd.DataFrame(data = DEFAULT_LATLONG,columns=['latitude','longitude'])
df_label['label'] = DEFAULT_DEST

In [39]:
import altair as alt
ch = alt.Chart(data = df_all_pts,width = 900,height=600).encode(x=alt.Longitude('longitude',scale=alt.Scale(zero=False)),
                                                                 y=alt.Latitude('latitude',scale=alt.Scale(zero=False)),color = 'leg:N' ).mark_point(filled=True)
ch_lab = alt.Chart(data=df_label,width = 900,height=600).encode(x=alt.Longitude('longitude',scale=alt.Scale(zero=False)),
                                                                 y=alt.Latitude('latitude',scale=alt.Scale(zero=False)),text = 'label:N' ).mark_text(size=15)

(ch+ch_lab).interactive()

alt.LayerChart(...)

In [36]:
df_label

,latitude,longitude,label
0,51.04191,-114.05572,1122 3 St SE
1,51.04239,-114.06853,1706 11 ave sw
2,51.05276,-114.05288,445 Marsh Rd NE
3,51.12546,-114.08387,504 Berkley Cres NW
4,51.05257,-114.04246,880 Center Ave NE
5,51.03027,-114.06479,111 25 Ave SW
6,51.02312,-114.06122,7016 34 Ave NW
7,51.03297,-114.02386,2313 15A st SE
